In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
import os.path

class main(object):
    
    def __init__(self, n_class, len_seq, dim_mfcc, batch_train, learning_rate, start_step, step_train, path_tfr_train, dir_log,
                 batch_test, test_step, step_test, path_tfr_test):
        self.n_class = n_class
        self.len_seq = len_seq
        self.dim_mfcc = dim_mfcc
        self.batch_train = batch_train
        self.learning_rate = learning_rate
        self.start_step = start_step
        self.step_train = step_train
        self.path_tfr_train = path_tfr_train
        self.dir_log = dir_log
        self.batch_test = batch_test
        self.test_step = test_step
        self.step_test = step_test
        self.path_tfr_test = path_tfr_test

    #TFRecordsからデータセット取り出し
    def input(self, path_tfr):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)

        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })

        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)

        datas = tf.reshape(datas, [self.len_seq, self.dim_mfcc])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.batch(
            [datas, labels],
            batch_size=self.batch_test, capacity=1000+self.batch_test*self.dim_mfcc
        )

        return datas, labels

    #TFRecordsからデータセット取り出し(shuffle)
    def input_shuffle(self, path_tfr):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)

        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })
        
        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)
        
        datas = tf.reshape(datas, [self.len_seq, self.dim_mfcc])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.shuffle_batch(
            [datas, labels],
            batch_size=self.batch_train, capacity=1000+self.batch_train*self.dim_mfcc,
            min_after_dequeue=1000
        )
            
        return datas, labels
    
    #dirで指定されたパスが存在しない場合ディレクトリ作成
    def make_dir(self,dir,format=False):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if format and os.path.exists(dir):
            shutil.rmtree(dir)

    #tensorboardのサマリに追加する
    def variable_summaries(self, var):
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)
        
    #重みベクトルを初期化して返す
    def variable(self, name, shape, stddev):
        var = tf.get_variable(name, shape=shape, initializer=tf.truncated_normal_initializer(stddev=stddev))
        return var
        
    #Linear
    def Linear(self, x, n_inputs, n_units, stddev, l_name):
        with tf.variable_scope(l_name) as scope:
            weights = self.variable('weights', shape=[n_inputs, n_units], stddev=stddev)
            biases = tf.get_variable('biases', shape=[n_units], initializer=tf.constant_initializer(0.0))
            linear = tf.nn.bias_add(tf.matmul(x, weights), biases, name=scope.name)
            self.variable_summaries(linear)
            return linear
        
    #LSTM
    def LSTM(self, x, n_units, l_name):
        with tf.variable_scope(l_name) as scope:
            lstm_cell = rnn.BasicLSTMCell(n_units, forget_bias=1.0)
            outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
            self.variable_summaries(outputs[-1])
            return outputs, states

    #Model
    def model(self, x):
        x_unstack = tf.unstack(x, self.len_seq, 1)
        x_unstack_rev = x_unstack[::-1]

        lstm1 = self.LSTM(x_unstack, n_units=256, l_name='lstm1')
        #lstm2 = self.LSTM(lstm1[0], n_units=256, l_name='lstm2')
        
        lstm_rev1 = self.LSTM(x_unstack_rev, n_units=256, l_name='lstm_rev1')
        #lstm_rev2 = self.LSTM(lstm_rev1[0], n_units=256, l_name='lstm_rev2')

        concatenate = tf.concat([lstm1[0][-1], lstm_rev1[0][-1]], axis=1)
        
        linear1 = self.Linear(concatenate, n_inputs=512, n_units=self.n_class, stddev=0.01, l_name='linear1')
        return linear1
        
    #トレーニング
    def train(self):
        sess = tf.InteractiveSession()
        
        x = tf.placeholder(tf.float32, shape=[None, self.len_seq, self.dim_mfcc])
        y_ = tf.placeholder(tf.float32, shape=[None, self.n_class])

        preds = self.model(x)
        
        with tf.name_scope('cross_entropy'):
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=preds, labels=y_))
            tf.add_to_collection('losses', cross_entropy)
            error=tf.add_n(tf.get_collection('losses'), name='total_loss')
            self.variable_summaries(error)

        with tf.name_scope('accuracy'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(error)
            correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            self.variable_summaries(accuracy)
            
        merged = tf.summary.merge_all()
        dir_log = os.path.join(self.dir_log, 'train')
        self.make_dir(dir_log)
        writer = tf.summary.FileWriter(dir_log, sess.graph)
        
        saver = tf.train.Saver(max_to_keep=1000)
        
        datas, labels = self.input_shuffle(self.path_tfr_train)
        labels = tf.one_hot(labels, depth=self.n_class, dtype=tf.float32)
        
        n_training_iters = self.step_train * self.batch_train
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        
        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            
            if(not self.start_step == 1):
                saver.restore(sess, os.path.join(self.dir_log, 'save_files/model.ckpt-'+str(self.start_step)))
            step = self.start_step
            while step * self.batch_train <= n_training_iters:
                batch = sess.run([datas, labels])
                batch[1] = batch[1].reshape([-1, self.n_class])
                sess.run(optimizer, feed_dict={x: batch[0], y_:batch[1]})
                if step % 100 == 0:
                    run_options  = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    run_metadata = tf.RunMetadata()
                    summary = sess.run(merged,
                        feed_dict={x: batch[0], y_:batch[1]},
                        options=run_options, run_metadata=run_metadata)
                    writer.add_summary(summary, step)
                    acc = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
                    loss = sess.run(cross_entropy, feed_dict={x: batch[0], y_: batch[1]})
                    print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
                    dir_ckpt = os.path.join(self.dir_log, 'save_files')
                    self.make_dir(dir_ckpt)
                    saver.save(sess, os.path.join(dir_ckpt, 'model.ckpt'), global_step=(step))
                step += 1

            coord.request_stop()
            coord.join(threads)
            writer.close()
            
    #テスト
    def test(self):
        sess = tf.InteractiveSession()

        x = tf.placeholder(tf.float32, shape=[None, self.len_seq, self.dim_mfcc])
        y_ = tf.placeholder(tf.float32, shape=[None, self.n_class])

        preds = self.model(x)

        with tf.name_scope('cross_entropy'):
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=preds, labels=y_))
            tf.add_to_collection('losses', cross_entropy)
            error=tf.add_n(tf.get_collection('losses'), name='total_loss')
            self.variable_summaries(error)

        with tf.name_scope('accuracy'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(error)
            correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            self.variable_summaries(accuracy)

        merged = tf.summary.merge_all()
        dir_log = os.path.join(self.dir_log, 'test')
        self.make_dir(dir_log)
        writer = tf.summary.FileWriter(dir_log, sess.graph)

        saver = tf.train.Saver(max_to_keep=1000)

        datas, labels = self.input(self.path_tfr_test)
        labels = tf.one_hot(labels, depth=self.n_class, dtype=tf.float32)

        n_test_iters = self.step_test * self.batch_test
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]

        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            saver.restore(sess, os.path.join(self.dir_log, 'save_files/model.ckpt-'+str(self.test_step)))
            step = 1
            acc_all = 0.0
            while step * self.batch_test <= n_test_iters:
                batch = sess.run([datas, labels])
                batch[1] = batch[1].reshape([-1, self.n_class])
                sess.run(optimizer, feed_dict={x: batch[0], y_:batch[1]})
                run_options  = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary = sess.run(merged,
                    feed_dict={x: batch[0], y_:batch[1]},
                    options=run_options, run_metadata=run_metadata)
                writer.add_summary(summary, step)
                acc = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
                loss = sess.run(cross_entropy, feed_dict={x: batch[0], y_: batch[1]})
                print('step: {} / loss: {:.6f} / test_acc: {:.5f}'.format(step, loss, acc))
                step += 1

            coord.request_stop()
            coord.join(threads)
            writer.close()

In [2]:
m = main(
    n_class = 22,
    len_seq = 300,
    dim_mfcc = 39,
    batch_train = 30,
    learning_rate = 0.001,
    start_step = 1,
    step_train = 3000,
    path_tfr_train = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset/train-male.tfrecords',
    dir_log = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/log',
    batch_test = 25,
    test_step = 300,
    step_test = 11,
    path_tfr_test = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset/test-male.tfrecords'
    )
m.train()

step: 100 / loss: 2.922967 / acc: 0.03333
step: 200 / loss: 2.051805 / acc: 0.46667
step: 300 / loss: 0.879652 / acc: 0.86667
step: 400 / loss: 0.198623 / acc: 1.00000
step: 500 / loss: 0.046468 / acc: 1.00000
step: 600 / loss: 0.028530 / acc: 1.00000
step: 700 / loss: 0.016011 / acc: 1.00000
step: 800 / loss: 0.009124 / acc: 1.00000
step: 900 / loss: 0.179624 / acc: 1.00000
step: 1000 / loss: 0.036639 / acc: 1.00000
step: 1100 / loss: 0.013676 / acc: 1.00000
step: 1200 / loss: 0.007700 / acc: 1.00000
step: 1300 / loss: 0.005904 / acc: 1.00000
step: 1400 / loss: 0.005123 / acc: 1.00000
step: 1500 / loss: 0.003529 / acc: 1.00000
step: 1600 / loss: 0.003411 / acc: 1.00000
step: 1700 / loss: 0.002500 / acc: 1.00000
step: 1800 / loss: 0.002675 / acc: 1.00000
step: 1900 / loss: 0.001484 / acc: 1.00000
step: 2000 / loss: 0.001909 / acc: 1.00000
step: 2100 / loss: 0.001524 / acc: 1.00000
step: 2200 / loss: 0.001440 / acc: 1.00000
step: 2300 / loss: 0.001200 / acc: 1.00000
step: 2400 / loss: 0